In [1]:
import cv2
import matplotlib.pyplot as plt
from glob import glob
import json
from PIL import Image
from tqdm import tqdm
import numpy as np

In [2]:
labels = glob('./dataset/BreastCancer_US_256/label/*')
len(labels)

1084

In [3]:
gt_boxes_dict = {}
for label in tqdm(labels):
    # if 'BUSI_imagesTr_benign_147' not in label:
    #     continue
    mask = Image.open(label)
    mask = np.array(mask)
    # print(mask.shape)
    bboxes = []
    ctrs, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    for ctr in ctrs:
        if len(ctr) < 40: continue
        minx, miny, maxx, maxy = min(ctr[:,0,0]), min(ctr[:,0,1]), max(ctr[:,0,0]), max(ctr[:,0,1])
        bboxes.append((int(minx), int(miny), int(maxx), int(maxy)))

    gt_boxes_dict[label.split('/')[-1].replace('.png', '')] = bboxes

100%|██████████| 1084/1084 [00:00<00:00, 1939.80it/s]


In [4]:
gt_boxes_dict

{'BUSI_imagesTr_benign_359': [(97, 76, 143, 98)],
 'BUSI_imagesTr_benign_36': [(85, 96, 181, 168)],
 'BUSI_imagesTr_malignant_36': [(71, 55, 190, 123)],
 'QAMEBI_imagesTr_48_Malignant': [(41, 39, 214, 254)],
 'BUSI_imagesTr_malignant_78': [(82, 52, 185, 117)],
 'UDIAT_imagesTr_000011': [(45, 54, 132, 87)],
 'QAMEBI_imagesTr_105_Benign': [(92, 101, 162, 149)],
 'BUSI_imagesTr_benign_82': [(133, 108, 191, 152)],
 'STU_imagesTr_14': [(94, 6, 169, 97)],
 'BUSI_imagesTr_malignant_93': [(70, 50, 204, 130)],
 'BUSI_imagesTr_benign_46': [(104, 76, 130, 97)],
 'BUSI_imagesTr_benign_415': [(26, 51, 250, 185)],
 'BUSI_imagesTr_benign_311': [(8, 19, 201, 84)],
 'BUSI_imagesTr_malignant_143': [(54, 78, 146, 119)],
 'BUSI_imagesTr_benign_435': [(86, 55, 122, 82)],
 'BUSI_imagesTr_benign_2': [(47, 123, 121, 157)],
 'STU_imagesTr_28': [(10, 46, 199, 149)],
 'BUSI_imagesTr_benign_357': [(76, 91, 147, 122)],
 'QAMEBI_imagesTr_8_Malignant': [(47, 30, 190, 182)],
 'BUSI_imagesTr_malignant_137': [(47, 10, 

In [5]:
with open(f'./configs/bbox-BreastCancer_256.json', 'w') as f:
    json.dump(gt_boxes_dict, f, indent = 2)

In [8]:
from torchmetrics.detection import MeanAveragePrecision

In [11]:
from torch import tensor
preds = [
dict(
    boxes=tensor([[258.0, 41.0, 606.0, 285.0]]),
    scores=tensor([0.536]),
    labels=tensor([0]),
    )
]
target = [
    dict(
        boxes=tensor([[214.0, 41.0, 562.0, 285.0]]),
        labels=tensor([0]),
        )
]

In [12]:
metric = MeanAveragePrecision(iou_type="bbox")
metric.update(preds, target)
from pprint import pprint
pprint(metric.compute())

{'classes': tensor(0, dtype=torch.int32),
 'map': tensor(0.6000),
 'map_50': tensor(1.),
 'map_75': tensor(1.),
 'map_large': tensor(0.6000),
 'map_medium': tensor(-1.),
 'map_per_class': tensor(-1.),
 'map_small': tensor(-1.),
 'mar_1': tensor(0.6000),
 'mar_10': tensor(0.6000),
 'mar_100': tensor(0.6000),
 'mar_100_per_class': tensor(-1.),
 'mar_large': tensor(0.6000),
 'mar_medium': tensor(-1.),
 'mar_small': tensor(-1.)}


In [10]:
from torchmetrics.detection import MeanAveragePrecision
import torch
pred_boxes = [
    {'boxes': torch.tensor([[52,70,181,181]]),
        'labels': torch.tensor([1]),
        'scores': torch.tensor([0.75])
    }
]
target_boxes = [
    {'boxes': torch.tensor([[8,66,144,185]]),
        'labels': torch.tensor([1]),
    }
]

metric = MeanAveragePrecision(iou_type="bbox")
metric.update(pred_boxes, target_boxes)
metric.compute()

{'map': tensor(0.1000),
 'map_50': tensor(1.),
 'map_75': tensor(0.),
 'map_small': tensor(-1.),
 'map_medium': tensor(-1.),
 'map_large': tensor(0.1000),
 'mar_1': tensor(0.1000),
 'mar_10': tensor(0.1000),
 'mar_100': tensor(0.1000),
 'mar_small': tensor(-1.),
 'mar_medium': tensor(-1.),
 'mar_large': tensor(0.1000),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor(1, dtype=torch.int32)}

In [2]:
with open('./configs/sec-32_cluster_centroid_6.json', 'r') as f:
    data = json.load(f)

In [31]:
def find_min_distance(mask, point):
    mask_coords = np.argwhere(mask == 1)
    distances = np.sqrt((mask_coords[:, 0] - point[0])**2 + (mask_coords[:, 1] - point[1])**2)
    return np.min(distances)

new_dicts = {}
for filename in tqdm(data):
    if filename != 'BUSI_imagesTr_benign_10':continue
    print(filename)
    points = data[filename]
    mask = cv2.imread(f'./dataset/BreastCancer_US_256/label/{filename}.png', 0)
    mask[mask!=0] = 1
    ctrs, _ = cv2.findContours((mask).astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if len(ctrs) == 1:
        ctr = ctrs[0]
        labels = []
        for point in points:
            pt = (int(point[0]), int(point[1])) # x, y
            if mask[pt[1], pt[0]] != 0:     # 내부에 존재하는가?
                labels.append(1)
            else:
                labels.append(0)
            print(pt, mask[pt[1], pt[0]], find_min_distance(mask, pt) < 20, labels)
    else:
        print(filename)
    # for ctr in ctrs:
    #     if len(ctr) < 40: continue
    #     for point in points:
    #         print(point)
    break

 60%|█████▉    | 645/1084 [00:00<00:00, 80985.66it/s]

BUSI_imagesTr_benign_10
(100, 100) 0 True [0]
(229, 152) 0 False [0, 0]
(57, 83) 1 True [0, 0, 1]
(164, 153) 0 False [0, 0, 1, 0]
(209, 69) 0 False [0, 0, 1, 0, 0]
(86, 160) 0 False [0, 0, 1, 0, 0, 0]
